In [60]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
train_tape = "Z:/rppg/ccnu_datatape_32x8x8_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_32x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [77]:
model = keras.Sequential([
    layers.Input(shape=(32, 8, 8, 3)),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(4, (2, 2, 2), (1, 2, 2), padding='same', activation='relu'),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(2, (2, 2, 2), (1, 2, 2), padding='same', activation='relu'),
    layers.LayerNormalization(axis=(1,)),
    layers.AvgPool3D((1, 2, 2)),
    layers.Conv3D(1, 1, 1),
    layers.Flatten(),
], name='NoobHeart')
model.compile(optimizer='adam', loss='mae')
print(f'Flops per frame:{get_flops(model, input_sig=[tf.TensorSpec([1, 32, 8, 8, 3])])/32:.0f}')
model.summary()

Flops per frame:5790
Model: "NoobHeart"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization_44 (Laye (None, 32, 8, 8, 3)       64        
_________________________________________________________________
conv3d_46 (Conv3D)           (None, 32, 4, 4, 4)       100       
_________________________________________________________________
layer_normalization_45 (Laye (None, 32, 4, 4, 4)       64        
_________________________________________________________________
conv3d_47 (Conv3D)           (None, 32, 2, 2, 2)       66        
_________________________________________________________________
layer_normalization_46 (Laye (None, 32, 2, 2, 2)       64        
_________________________________________________________________
average_pooling3d_15 (Averag (None, 32, 1, 1, 2)       0         
_________________________________________________________________
conv3d_48 (Conv3D)           (None, 

In [72]:
model.fit(train.shuffle(2000).batch(128), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/NoobHeart.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
model.load_weights('../weights/NoobHeart.h5')

Epoch 1/10
1189/1189 [==============================] - 6s 5ms/step - loss: 0.7876 - val_loss: 0.7506

Epoch 00001: val_loss improved from inf to 0.75062, saving model to ../weights\NoobHeart.h5
Epoch 2/10
1189/1189 [==============================] - 5s 4ms/step - loss: 0.7425 - val_loss: 0.7400

Epoch 00002: val_loss improved from 0.75062 to 0.73999, saving model to ../weights\NoobHeart.h5
Epoch 3/10
1189/1189 [==============================] - 5s 4ms/step - loss: 0.7351 - val_loss: 0.7267

Epoch 00003: val_loss improved from 0.73999 to 0.72669, saving model to ../weights\NoobHeart.h5
Epoch 4/10
1189/1189 [==============================] - 5s 4ms/step - loss: 0.7259 - val_loss: 0.7059

Epoch 00004: val_loss improved from 0.72669 to 0.70590, saving model to ../weights\NoobHeart.h5
Epoch 5/10
1189/1189 [==============================] - 5s 4ms/step - loss: 0.7217 - val_loss: 0.7006

Epoch 00005: val_loss improved from 0.70590 to 0.70059, saving model to ../weights\NoobHeart.h5
Epoch 6/1

In [26]:
model.load_weights('../weights/NoobHeart.h5')

In [76]:
eval_on_dataset(test_set_CCNU, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_CCNU.h5')
get_metrics('../results/NoobHeart_CCNU_CCNU.h5',)

100%|██████████| 179/179 [00:14<00:00, 12.40it/s]


{'Sliding window': {'MAE': 1.784, 'RMSE': 5.851, 'R': 0.8307},
 'Whole video': {'MAE': 0.961, 'RMSE': 2.933, 'R': 0.95338}}

In [75]:
eval_on_dataset(test_set_CCNU_rPPG, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_CCNU_rPPG.h5')
get_metrics('../results/NoobHeart_CCNU_CCNU_rPPG.h5',)

100%|██████████| 54/54 [00:03<00:00, 16.06it/s]


{'Sliding window': {'MAE': 1.459, 'RMSE': 4.303, 'R': 0.90202},
 'Whole video': {'MAE': 0.825, 'RMSE': 1.471, 'R': 0.98854}}

In [74]:
eval_on_dataset(test_set_PURE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_PURE.h5')
get_metrics('../results/NoobHeart_CCNU_PURE.h5',)

100%|██████████| 59/59 [00:02<00:00, 20.82it/s]


{'Sliding window': {'MAE': 0.95, 'RMSE': 4.404, 'R': 0.98178},
 'Whole video': {'MAE': 0.475, 'RMSE': 0.794, 'R': 0.99952}}

In [73]:
eval_on_dataset(test_set_UBFC_rPPG2, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_UBFC.h5')
get_metrics('../results/NoobHeart_CCNU_UBFC.h5',)

100%|██████████| 42/42 [00:01<00:00, 22.36it/s]


{'Sliding window': {'MAE': 1.873, 'RMSE': 4.465, 'R': 0.96794},
 'Whole video': {'MAE': 1.268, 'RMSE': 1.846, 'R': 0.99561}}